In [3]:
# from langchain.chat_models import ChatOpenAI

# llm = OpenAI(model_name="gpt-3.5-turbo-1106") 
## davinci-3 is deprecated, so I need to add  - model_name="gpt-3.5-turbo-1106"-
# chat = ChatOpenAI()

# a = llm.predict("How mant planets are there?")
# b = chat.predict("How mant planets are there?")

# b


In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1 # set how this model is creative
)



In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geogrphy expert. And you are only reply in {language}",
    ),
    AIMessage(
        content="Ciao, mi chiamo {name}!",
    ),
    HumanMessage(
        content="What is the distance between {country_a} and {country_b}. Also, what is your name?",
    ),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Mi chiamo Paolo, sono un esperto di geografia. Posso aiutarti con altre domande?')